In [2]:
!pip install imblearn
import os
import rasterio
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC 
from sklearn.metrics import classification_report, confusion_matrix 
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV



In [3]:
train_files = ['tile00.csv', 'tile01.csv', 'tile02.csv' ,'tile03.csv']
train = pd.concat([pd.read_csv(f) for f in train_files], ignore_index=True)
X = train.loc[:, train.columns != 'class']
y = train[['class']]

X = X.rename(columns={x:y for x,y in zip(X.columns,range(0,len(X.columns)))})
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.3, random_state=42)

/tmp/ipykernel_1584/2016119411.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(0, inplace=True)


In [3]:
clf = BalancedBaggingClassifier(estimator=DecisionTreeClassifier(), sampling_strategy = 'not majority', replacement=False, n_estimators=10, random_state=0, verbose=1).fit(X_train, y_train)
predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00   3138409
         1.0       0.00      0.00      0.00       487
         2.0       0.00      0.00      0.00       279
         3.0       1.00      1.00      1.00         1
         4.0       0.00      0.00      0.00       185
         5.0       0.00      0.00      0.00        65
         6.0       0.00      0.00      0.00        46
         7.0       1.00      0.10      0.18        10

    accuracy                           1.00   3139482
   macro avg       0.37      0.26      0.27   3139482
weighted avg       1.00      1.00      1.00   3139482



/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [6]:


clf = BaggingClassifier(estimator=DecisionTreeClassifier(),  n_estimators=10, random_state=0, verbose=1).fit(X_train, y_train)
bag_preds = clf.predict(X_test)

print(classification_report(y_test, bag_preds))

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00   3138409
         1.0       0.77      0.31      0.44       487
         2.0       0.81      0.25      0.39       279
         3.0       0.00      0.00      0.00         1
         4.0       0.69      0.26      0.38       185
         5.0       0.72      0.35      0.47        65
         6.0       0.79      0.24      0.37        46
         7.0       0.00      0.00      0.00        10

    accuracy                           1.00   3139482
   macro avg       0.60      0.30      0.38   3139482
weighted avg       1.00      1.00      1.00   3139482



/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [4]:
param_grid = {
    'base_estimator__max_depth' : [1, 2, 3, 4, 5],
    'max_samples' : [0.05, 0.1, 0.2, 0.5]
}

clf = GridSearchCV(BaggingClassifier(DecisionTreeClassifier(), bootstrap = True, 
                                     n_estimators = 10, max_features = 0.5), param_grid)
clf.fit(X_train, y_train)

/home/codespace/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:717: FutureWarning: Parameter 'base_estimator' of BaggingClassifier is deprecated in favor of 'estimator'. See BaggingClassifier's docstring for more details.
  estimator = estimator.set_params(**clone(parameters, safe=False))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:717: FutureWarning: Parameter 'base_estimator' of BaggingClassifier is deprecated in favor of 'estimator'. See BaggingClassifier's docstring for more details.
  estimator = estimator.set_params(**clone(parameters, safe=False))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:717: FutureWarning: Parameter 'base_estimator' of BaggingClassifier is deprecated in favor of 'estimator'. See BaggingClassifier's docstring for more details.
  estimator = estimator.set_params(**clone(parameters, safe=False))
/home/codespace/.local/lib/python3.10/site-pac

GridSearchCV(estimator=BaggingClassifier(estimator=DecisionTreeClassifier(),
                                         max_features=0.5),
             param_grid={'base_estimator__max_depth': [1, 2, 3, 4, 5],
                         'max_samples': [0.05, 0.1, 0.2, 0.5]})

In [5]:
ft_bag_preds = clf.predict(X_test)

print(classification_report(y_test, ft_bag_preds))

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00   3138409
         1.0       0.00      0.00      0.00       487
         2.0       0.00      0.00      0.00       279
         3.0       0.00      0.00      0.00         1
         4.0       0.00      0.00      0.00       185
         5.0       0.00      0.00      0.00        65
         6.0       0.00      0.00      0.00        46
         7.0       0.00      0.00      0.00        10

    accuracy                           1.00   3139482
   macro avg       0.12      0.12      0.12   3139482
weighted avg       1.00      1.00      1.00   3139482



/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
